In [ ]:
!pip install -r requirements.txt

In [2]:
# Import the libraries
import cv2
import mediapipe as mp
import pyautogui as pt
import numpy as np

In [3]:
# Function to check if two points are close
def is_point_close(pt1, pt2):
    return ((pt1[0] - pt2[0]) ** 2 + (pt1[1] - pt2[1]) ** 2) ** 0.5 

In [4]:
def create_gradient(start_color, end_color, size):
    start_color = np.array(start_color, dtype=np.uint8)
    end_color = np.array(end_color, dtype=np.uint8)
    return np.array([np.linspace(start_color[i], end_color[i], size[1]) for i in range(3)], dtype=np.uint8).T

In [5]:
# Function to check if a point is inside a rectangle
def is_point_inside_rect(pt, rect):
    x, y, w, h = rect
    return x <= pt[0] <= x + w and y <= pt[1] <= y + h

In [6]:
# Function to draw a rectangle around the key
def draw_key_rect(image, key_rect, start_color=(100, 255, 0), end_color=(255, 0, 0), thickness=-1):
    x, y, w, h = key_rect
    gradient = create_gradient(start_color, end_color, (h, w))
    if thickness == -1:
        image_slice = image[y:y+h, x:x+w]
        if image_slice.shape[:2] == gradient.shape:
            image[y:y+h, x:x+w] = cv2.addWeighted(image_slice, 0.5, cv2.cvtColor(gradient, cv2.COLOR_RGB2BGR), 0.5, 0)
    else:
        cv2.rectangle(image, (x, y), (x + w, y + h), end_color, thickness)

In [7]:
# Main function
def main():
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands()
    cap = cv2.VideoCapture(0)
    keys = ['q', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p',
        'a', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l', '"',
        'z', 'x', 'c', 'v', 'b', 'n', 'm', ',', '.', '?']
    key_coords = {}

    # Define the start positions and the step for x and y
    x_start, x_step = 15, 60
    y_start, y_step = 15, 60

    # Define the width and height of the keys
    width, height = 50, 50

    # Define the number of keys per row
    keys_per_row = [10, 10, 10]

    # Generate the keys and their rectangles
    key_index = 0
    for row, k in enumerate(keys_per_row):
        for i in range(k):
            key = keys[key_index]
            x = x_start + i * x_step
            y = y_start + row * y_step
            key_coords[key] = (x, y, width, height)
            key_index += 1

    # keys is a dictionary that maps a key name to a tuple of (x, y, width, height)
    keys = {'capslock': (x_start, y_start + 3 * y_step, width*3, height), 
        ' ': (int(x_start + width*3.3), y_start + 3 * y_step, width*2, height), 
        'Enter': (int(x_start + width*5.3), y_start + 3 * y_step, width*2, height), 
        'backspace': (int(x_start + width*7.3), y_start + 3 * y_step, width*4, height),
        'up': (x_start + 4 * width, y_start + 4 * y_step, width, height),
        'down': (x_start + 5 * width, y_start + 4 * y_step, width, height),
        'left': (x_start + 6 * width, y_start + 4 * y_step, width, height),
        'right': (x_start + 7 * width, y_start + 4 * y_step, width, height),
        'esc': (x_start + 5 * width, y_start + 5 * y_step, width * 2, height)}
    key_coords.update(keys)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Flip the frame horizontally for a later selfie-view display
        frame = cv2.flip(frame, 1)

        # Convert the BGR image to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with Mediapipe hands
        result = hands.process(rgb_frame)

        # Draw the keys
        for key, key_rect in key_coords.items():
            draw_key_rect(frame, key_rect, start_color=(255, 182, 193), end_color=(255, 105, 180), thickness=-1)
            cv2.putText(frame, key, (key_rect[0] + 10, key_rect[1] + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

        # Check if hands are detected
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                # Get the position of the index finger tip
                index_finger_tip = (
                    hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * frame.shape[1],
                    hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * frame.shape[0]
                )
                middle_finger_tip = (
                    hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x * frame.shape[1],
                    hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y * frame.shape[0]
                )
                distance = is_point_close(index_finger_tip, middle_finger_tip)

                # Check if the distance is close to zero
                if distance < 15:  # you can adjust this threshold as needed
                    for key, key_rect in key_coords.items():
                        if is_point_inside_rect(index_finger_tip, key_rect):
                            # Draw the key rectangle and label
                            # draw_key_rect(frame, key_rect, color=(0, 255, 0), thickness=1)
                            draw_key_rect(frame, key_rect, start_color=(25, 68, 0), end_color=(255, 0, 0), thickness=1)
                            cv2.putText(frame, key, (key_rect[0] + 10, key_rect[1] + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
                            pt.press(key, interval=0.03)

        # Resize the frame
        frame = cv2.resize(frame, (1280, 720))
        # Display the resulting frame
        cv2.imshow('Virtual Keyboard', frame)

        # Break the loop when 'ESC' key is pressed
        if cv2.waitKey(1) == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

In [8]:
if __name__ == "__main__":
    main()